# Data Aggregation and Group Operations

Categorizing a data set and applying a function to each group, whether an aggregation or transformation, is often a critical component of a data analysis workflow.

pandas provides a flexible and high-performance groupby facility, enabling you to slice and dice, and summarize data sets in a natural way.

- Split a pandas object into pieces using one or more keys
- Computing group summary statistics
- Apply a varying set of functions to each column of a DataFrame
- Apply within-group transformations or other manipulations
- Compute pivot tables and cross-tabulations
- Perform quantile analysis and other data-derived group analyses

In [1]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from numpy import nan as NA

## GroupBy Mechanics

*split-apply-combine*

- In the first stage of the process, data contained in a pandas object, whether a Series, DataFrame, or otherwise, is split into groups based on one or more keys that you provide. The splitting is performed on a particular axis of an object.
- A function is applied to each group, producing a new value.
- Finally, the results of all those function applications are combined into a result object.

<img src='./Pics/picture_5_1.png'>

Each grouping key can take many forms, and the keys do not have to be all of the same type:
- A list or array of values that is the same length as the axis being grouped
- A value indicating a column name in a DataFrame
- A dict or Series giving a correspondence between the values on the axis being grouped and the group names
- A function to be invoked on the axis index or the individual labels in the index

In [ ]:
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

Suppose you wanted to compute the mean of the **data1** column using the groups labels from **key1**. One is to access **data1** and call **groupby** with the column (a Series) at **key1**:

In [ ]:
grouped = df['data1'].groupby(df['key1'])
grouped

This **grouped** variable is now a *GroupBy* object. It has not actually computed anything yet except for some intermediate data about the group key **df['key1']**. The idea is that this object has all of the information needed to then apply some operation to each of the groups.

In [ ]:
grouped.mean()

The data (a Series) has been aggregated according to the group key, producing a new Series that is now indexed by the unique values in the **key1** column. The result index has the name **'key1'** because the DataFrame column **df['key1']** did.

If instead we had passed multiple arrays as a list,

In [ ]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

We grouped the data using two keys, and the resulting Series now has a hierarchical index consisting of the unique pairs of keys observed:

In [ ]:
means.unstack()

The group keys are all Series, though they could be any arrays of the right length:

In [ ]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio']);
years = np.array([2005, 2005, 2006, 2005, 2006])

In [ ]:
df

In [ ]:
df['data1'].groupby([states, years]).mean()

Frequently the grouping information to be found in the same DataFrame as the data we want to work on. In that case, we can pass column names as the group keys:

In [ ]:
df.groupby('key1').mean()

In [ ]:
df.groupby(['key1', 'key2']).mean()

We may have noticed in the first case **df.groupby('key1').mean()** that there is no **key2** column in the result. Because **df['key2']** is not numeric data, it is said to be a *nuisance column*, which is therefore excluded from the result. By default, all of the numeric columns are aggregated.

Regardless of the objective in using **groupby**, a generally useful GroupBy method is **size** which return a Series containing group sizes:

In [ ]:
 df.groupby(['key1', 'key2']).size()

*GroupBy Mechanics* >
### Iterating Over Groups

The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data.

In [ ]:
for name, group in df.groupby('key1'):
    print(name)
    print(type(group))
    print(group)

In the case of multiple keys, the first element in the tuple will be a tuple of key values:

In [ ]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(k1, k2)
    print(group)

We can choose to do whatever we want with the pieces of data. A recipe we may find useful is computing a dict of the data pieces as a one-liner:

In [ ]:
pieces = dict(list(df.groupby('key1')))
pieces

In [ ]:
pieces['b']

By default **groupby** groups on axis=0, but you can group on any of the other axes.

In [ ]:
df.dtypes

In [ ]:
grouped = df.groupby(df.dtypes, axis=1)

In [ ]:
dict(list(grouped))

*GroupBy Mechanics* >
### Selecting a Column or Subset of Columns

Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of *selecting those columns* for aggregation.

In [ ]:
df.groupby('key1')['data1']

In [ ]:
df['data1'].groupby(df['key1'])

Especially for large data sets, it may be desirable to aggregate only a few.

In [ ]:
df.groupby(['key1', 'key2'])[['data2']].mean()

The object returned by this indexing operation is a grouped DataFrame if a list or array is passed and a grouped Series is just a single column name that is passed as a scalar:

In [ ]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [ ]:
dict(list(s_grouped))

In [ ]:
s_grouped.mean()

*GroupBy Mechanics* >
### Grouping with Dicts and Series

Grouping information may exist in a form other than an array.

In [ ]:
people = DataFrame(np.random.randn(5, 5), 
                   columns=['a', 'b', 'c', 'd', 'e'], 
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

In [ ]:
people.loc[2:3, ['b', 'c']] = np.nan # Add a few NA values

In [ ]:
people

In [ ]:
people.sum(axis=1)

We have a group correspondence for the columns and want to sum together the columns by group:

In [ ]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

We could easily construct an array from this dict to pass to **groupby**, but instead we can just pass the dict:

In [ ]:
by_column = people.groupby(mapping, axis=1)

In [ ]:
dict(list(by_column))

In [ ]:
by_column.sum()

The same functionality holds for Series, which can be viewed as a fixed size mapping. When we used Series as group keys in the above examples, pandas does, in fact, inspect each Series to ensure that its index is aligned with the axis it’s grouping:

In [ ]:
map_series = Series(mapping)
map_series

In [ ]:
people.groupby(map_series, axis=1).count()

*GroupBy Mechanics* >
### Grouping with Functions

Using Python functions in what can be fairly creative ways is a more abstract way of defining a group mapping compared with a dict or Series. Any function passed as a group key will be called once per index value, with the return values being used as the group names.

Suppose we wanted to group by the length of the names; we could compute an array of string lengths, but instead we can just pass the **len** function:

In [ ]:
people.groupby(len).sum()

Mixing functions with arrays, dicts, or Series is not a problem as everything gets converted to arrays internally:

In [ ]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [ ]:
people.groupby([len, key_list]).min()

*GroupBy Mechanics* >
### Grouping by Index Levels

A final convenience for hierarchically-indexed data sets is the ability to aggregate using one of the levels of an axis index. To do this, pass the level number or name using the level keyword:

In [ ]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]], names=['cty', 'tenor'])

In [ ]:
hier_df = DataFrame(np.random.randn(4, 5), columns=columns)

In [ ]:
hier_df

In [ ]:
hier_df.groupby(level='tenor', axis=1).count()

## Data Aggregation

By aggregation, It’s generally referred to any data transformation that produces scalar values from arrays. In the examples above we have used several of them, such as **mean**, **count**, **min** and **sum**.

Many common aggregations have optimized implementations that compute the statistics on the dataset in place. However, we are not limited to only this set of methods.

We can use aggregations of our own devising and additionally call any method that is also defined on the grouped object.

In [ ]:
df

In [ ]:
grouped = df.groupby('key1')

In [ ]:
grouped['data1'].quantile(0.9)

While **quantile** is not explicitly implemented for GroupBy, it is a Series method and thus available for use. Internally, GroupBy efficiently slices up the Series, calls **piece.quantile(0.9)** for each piece, then assembles those results together into the result object.

To use our own aggregation functions, pass any function that aggregates an array to the aggregate or agg method:

In [ ]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [ ]:
grouped.agg(peak_to_peak)

We’ll notice that some methods like describe also work, even though they are not aggregations, strictly speaking:

In [ ]:
grouped.describe()

<img src='./Pics/picture_5_2.png'>

*Data Aggregation* >
### Column-wise and Multiple Function Application

We may want to aggregate using a different function depending on the column or multiple functions at once. Fortunately, this is straightforward to do, which is illustrated through a number of examples.

In [ ]:
tips = pd.read_csv('./Data/tips.csv')
tips

In [ ]:
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [ ]:
tips[:6]

We can pass the name of the function as a string:

In [ ]:
grouped = tips.groupby(['sex', 'smoker'])

In [ ]:
grouped_pct = grouped['tip_pct']

In [ ]:
grouped_pct.mean()

In [ ]:
grouped_pct.agg('mean')

If we pass a list of functions or function names instead, we get back a DataFrame with column names taken from the functions:

In [ ]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

We don’t need to accept the names that GroupBy gives to the columns; notably lambda functions have the name '<**lambda**\>' which make them hard to identify.

In [ ]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

With a DataFrame, you have more options as you can specify a list of functions to apply to all of the columns or different functions per column.

In [ ]:
functions = ['count', 'mean', 'max']

In [ ]:
result = grouped['tip_pct', 'total_bill'].agg(functions)

In [ ]:
result

The resulting DataFrame has hierarchical columns, the same as you
would get aggregating each column separately and using concat to glue the results together using the column names as the keys argument:

In [ ]:
result['tip_pct']

A list of tuples with custom names can be passed:

In [ ]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [ ]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

Suppose we wanted to apply potentially different functions to one or more of the columns.

In [ ]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

In [ ]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'], 
             'size' : 'sum'})

*Data Aggregation* >
### Returning Aggregated Data in “unindexed” Form

The aggregated data comes back with an index, potentially hierarchical, composed from the unique group key combinations observed.

We can disable this behavior in most cases by passing **as_index=False** to **groupby**:

In [ ]:
tips.groupby(['sex', 'smoker'], as_index=False).mean()

## Group-wise Operations and Transformations

Aggregation is only one kind of group operation. It is a special case in the more general class of data transformations; that is, it accepts functions that reduce a one-dimensional array to a scalar value. 
Introduce to the **transform** and **apply** methods, which will enable us to do many other kinds of group operations.

In [ ]:
df

In [ ]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')

In [ ]:
k1_means

In [ ]:
pd.merge(df, k1_means, left_on='key1', right_index=True)

This works, but is somewhat inflexible. We can think of the operation as transforming the two data columns using the **np.mean** function. Let’s look back at the **people** Data-
Frame from earlier in the chapter and use the **transform** method on GroupBy:

In [ ]:
key = ['one', 'two', 'one', 'two', 'one']

In [ ]:
people

In [ ]:
people.groupby(key).mean()

In [ ]:
people.groupby(key).transform(np.mean)

**transform** applies a function to each group, then places the results in the appropriate locations. If each group produces a scalar value, it will be propagated. 

Suppose instead we wanted to subtract the mean value from each group. To do this, create a demeaning function and pass it to transform:

In [ ]:
def demean(arr):
    return arr - arr.mean()

In [ ]:
demeaned = people.groupby(key).transform(demean)

In [ ]:
demeaned

We can check that demeaned now has zero group means:

In [ ]:
demeaned.groupby(key).mean()

*Group-wise Operations and Transformations* >
### Apply: General split-apply-combine

**transform** is a more specialized function having rigid requirements: the passed function must either produce a scalar value to be broadcasted (like **np.mean**) or a transformed array of the same size.

The most general purpose GroupBy method is **apply**. **apply** splits the object being manipulated into pieces, invokes the passed function on each piece, then attempts to concatenate the pieces together.

Suppose we wanted to select the top five **tip_pct** values by group. First, it’s straightforward to write a function that selects the rows with the largest values in a particular column:

In [ ]:
def top(df, n=5, column='tip_pct') :
    return df.sort_values(by=column)[-n:]

In [ ]:
top(tips, n=5)

If we group by smoker, say, and call apply with this function, we get the following:

In [ ]:
tips.groupby('smoker').apply(top)

The **top** function is called on each piece of the DataFrame,
then the results are glued together using **pandas.concat**, labeling the pieces with the group names.

If we pass a function to apply that takes other arguments or keywords, we can pass these after the function:

In [ ]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

We may recall above **describe** on a GroupBy object:

In [ ]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [ ]:
result

In [ ]:
result.unstack('smoker')

*Group-wise Operations and Transformations* > *Apply: General split-apply-combine* >
#### Suppressing the group keys

We see that the resulting object has a hierarchical index formed
from the group keys along with the indexes of each piece of the original object. This can be disabled by passing **group_keys=False** to **groupby**:

In [ ]:
tips.groupby('smoker', group_keys=False).apply(top)

*Group-wise Operations and Transformations* >
### Quantile and Bucket Analysis

pandas has some tools, in particular **cut** and **qcut**, for slicing data up into buckets with bins of your choosing or by sample quantiles. Combining these functions with **groupby**, it becomes very simple to perform bucket or quantile analysis on a data set.

In [ ]:
frame = DataFrame({'data1': np.random.randn(1000), 
                   'data2': np.random.randn(1000)})

In [ ]:
factor = pd.cut(frame.data1, 4)
factor

In [ ]:
factor[:10]

The **Factor** object returned by **cut** can be passed directly to **groupby**. So we could compute a set of statistics for the **data2** column like so:

In [ ]:
def get_stats(group) :
    return {'min': group.min(), 'max': group.max(),
           'count': group.count(), 'mean': group.mean()}

In [ ]:
grouped = frame.data2.groupby(factor)

In [ ]:
grouped.apply(get_stats).unstack()

These were equal-length buckets; to compute equal-size buckets based on sample quantiles, use **qcut** and pass **labels=False** to just get quantile numbers.

In [ ]:
grouping = pd.qcut(frame.data1, 10, labels=False)

In [ ]:
grouped = frame.data2.groupby(grouping)

In [ ]:
grouped.apply(get_stats).unstack()

*Group-wise Operations and Transformations* >
### Example: Filling Missing Values with Group-specific Values

When cleaning up missing data, in some cases you will filter out data observations using **dropna**, but in others you may want to impute (fill in) the NA values using a fixed value or some value derived from the data. **fillna** is the right tool to use;

In [ ]:
s = Series(np.random.randn(6))

In [ ]:
s[::2] = np.nan

In [ ]:
s

In [ ]:
s.mean()

In [ ]:
s.fillna(s.mean())

Suppose we need the fill value to vary by group. As we may guess, we need only group the data and use apply with a function that calls **fillna** on each data chunk.

In [ ]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 
          'Oregon', 'Nevada', 'California', 'Idaho']

In [ ]:
group_key = ['East']*4 + ['West']*4

In [ ]:
group_key

In [ ]:
data = Series(np.random.randn(8), index=states)

In [ ]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [ ]:
data

In [ ]:
data.groupby(group_key).mean()

We can fill the NA values using the group means like so:

In [ ]:
fill_mean = lambda g : g.fillna(g.mean())

In [ ]:
data.groupby(group_key).apply(fill_mean)

We might have pre-defined fill values in our code that vary by group. Since the groups have a name attribute set internally, we can use that:

In [ ]:
fill_values = {'East': 0.5, 'West': -1}

In [ ]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [ ]:
data.groupby(group_key).apply(fill_func)

*Group-wise Operations and Transformations* >
### Example: Random Sampling and Permutation

Suppose we wanted to draw a random sample (with or without replacement) from a large dataset for Monte Carlo simulation purposes or some other application. There are a number of ways to perform the “draws”;

One way is to select the first **K** elements of  **np.random.permutation(N)**, where **N** is the size of your complete dataset and **K** the desired sample size.

In [ ]:
suits = ['H', 'S', 'C', 'D']

In [ ]:
card_val = (list(range(1, 11)) + [10]*3)*4

In [ ]:
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']

In [ ]:
cards = []

In [ ]:
for suit in ['H', 'S', 'C', 'D']: 
    cards.extend(str(num) + suit for num in base_names)

In [ ]:
deck = Series(card_val, index=cards)

So now we have a Series of length 52 whose index contains card names and values are the ones used in blackjack and other games:

In [ ]:
deck[:13]

Based on what described above, draw a hand of 5 cards from the desk.

In [ ]:
def draw(deck, n=5) :
    return deck.take(np.random.permutation(len(deck))[:n])

In [ ]:
draw(deck)

Suppose we wanted two random cards from each suit.

In [ ]:
get_suit = lambda card: card[-1]

In [ ]:
deck.groupby(get_suit).apply(draw, n=2)

In [ ]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

*Group-wise Operations and Transformations* >
### Example: Group Weighted Average and Correlation

Under the split-apply-combine paradigm of **groupby**, operations between columns in a DataFrame or two Series, such a group weighted average, become a routine affair.

In [ ]:
df = DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'], 
                'data': np.random.randn(8),
                'weights': np.random.rand(8)})

In [ ]:
df

The group weighted average by category would then be:

In [ ]:
grouped = df.groupby('category')

In [ ]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [ ]:
grouped.apply(get_wavg)

Consider a data set from Yahoo! Finance containing end of day prices for a few stocks and the S&P 500 index (the SPX ticker):

In [ ]:
close_px = pd.read_csv('./Data/stock_px.csv', parse_dates=True, 
                       index_col=0)

In [ ]:
close_px.info()

In [ ]:
close_px[-4:]

One task of interest might be to compute a DataFrame consisting of the yearly correlations of daily returns (computed from percent changes) with SPX.

In [ ]:
rets = close_px.pct_change().dropna()

In [ ]:
rets.info()

In [ ]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [ ]:
by_year = rets.groupby(lambda x: x.year)

In [ ]:
by_year.apply(spx_corr)

In [ ]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

*Group-wise Operations and Transformations* >
### Example: Group-wise Linear Regression

We can use groupby to perform more complex group-wise statistical analysis, as long as the function returns a pandas object or scalar value. For example, we can define the following **regress** function (using the **statsmodels** econometrics library) which executes an ordinary least squares (OLS) regression on each chunk of data:

In [ ]:
import statsmodels.api as sm 

In [ ]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars] 
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [ ]:
by_year.apply(regress, 'AAPL', ['SPX'])

## Pivot Tables and Cross-Tabulation

A *pivot* table is a data summarization tool frequently found in spreadsheet programs and other data analysis software. It aggregates a table of data by one or more keys, arranging the data in a rectangle with some of the group keys along the rows and some along the columns. 

Pivot tables in Python with pandas are made possible using the **groupby** facility combined with reshape operations utilizing hierarchical indexing.

DataFrame has a **pivot_table** method, and additionally there is a top-level **pandas.pivot_table** function. 

In addition to providing a convenience interface to **groupby**, **pivot_table** also can add partial totals, also known as *margins*.

We wanted to compute a table of group means (the default **pivot_table** aggregation type) arranged by **sex** and **smoker** on the rows:

In [ ]:
tips.pivot_table(index = ['sex', 'smoker'])

This could have been easily produced using **groupby**. Now, suppose we want to aggregate only **tip_pct** and **size**, and additionally group by **day**. We’ll put **smoker** in the table columns and **day** in the rows:

In [ ]:
tips.pivot_table(['tip_pct', 'size'],
                index = ['sex', 'day'],
                columns = 'smoker')

This table could be augmented to include partial totals by passing **margins=True**. This has the effect of adding **All** row and column labels, with corresponding values being the group statistics for all the data within a single tier.

In [ ]:
tips.pivot_table(['tip_pct', 'size'],
                index = ['sex', 'day'],
                columns = 'smoker', margins = True)

To use a different aggregation function, pass it to **aggfunc**.

In [ ]:
tips.pivot_table('tip_pct', index = ['sex', 'smoker'], columns = 'day', 
                aggfunc = len, margins = True)

If some combinations are empty (or otherwise NA), we may wish to pass a **fill_value**:

In [ ]:
tips.pivot_table('size', index = ['time', 'sex', 'smoker'], columns = 'day', 
                aggfunc = 'sum')

In [ ]:
tips.pivot_table('size', index = ['time', 'sex', 'smoker'], columns = 'day', 
                aggfunc = 'sum', fill_value = 0)

<img src='./Pics/picture_5_3.png'>

*Pivot Tables and Cross-Tabulation* >
### Cross-Tabulations: Crosstab

A cross-tabulation (or *crosstab* for short) is a special case of a pivot table that computes group frequencies. 

In [ ]:
cross_data = {'Sample': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
              'Gender': ['Female', 'Male', 'Female', 'Male', 'Male', 'Male', 'Female', 'Female', 'Male', 'Female'], 
              'Handedness': ['Right-handed', 'Left-handed', 'Right-handed', 'Right-handed', 'Left-handed', 
                             'Right-handed', 'Right-handed', 'Left-handed', 'Right-handed', 'Right-handed']}

In [ ]:
data = DataFrame(cross_data)

In [ ]:
data

In [ ]:
pd.crosstab(data.Gender, data.Handedness, margins=True)

The first two arguments to crosstab can each either be an array or Series or a list of arrays.

In [ ]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

## Example: 2012 Federal Election Commission Database

The US Federal Election Commission publishes data on contributions to political campaigns. This includes contributor names, occupation and employer, address, and contribution amount. An interesting dataset is from the 2012 US presidential election (http://www.fec.gov/disclosurep/PDownload.do).

In [ ]:
fec = pd.read_csv('./Data/P00000001-ALL.csv', low_memory=False)

In [ ]:
fec.info()

In [ ]:
fec.loc[123456]

We can see that there are no political party affiliations in the data, so this would be useful to add. We can get a list of all the unique political candidates using **unique**.

In [ ]:
unique_cands = fec.cand_nm.unique()

In [ ]:
unique_cands

In [ ]:
unique_cands[2]

In [ ]:
parties = {'Bachmann, Michelle': 'Republican', 
           'Cain, Herman': 'Republican',
           'Gingrich, Newt': 'Republican', 
           'Huntsman, Jon': 'Republican', 
           'Johnson, Gary Earl': 'Republican', 
           'McCotter, Thaddeus G': 'Republican', 
           'Obama, Barack': 'Democrat',
           'Paul, Ron': 'Republican',
           'Pawlenty, Timothy': 'Republican',
           'Perry, Rick': 'Republican',
           "Roemer, Charles E. 'Buddy' III": 'Republican', 
           'Romney, Mitt': 'Republican', 
           'Santorum, Rick': 'Republican'}

Using this mapping and the map method on Series objects, you can compute an array of political parties from the candidate names:

In [ ]:
fec.cand_nm[123456:123461]

In [ ]:
fec.cand_nm[123456:123461].map(parties)

In [ ]:
fec['party'] = fec.cand_nm.map(parties)

In [ ]:
fec['party'].value_counts()

A couple of data preparation points. First, this data includes both contributions and refunds (negative contribution amount):

In [ ]:
(fec.contb_receipt_amt > 0).value_counts()

A couple of data preparation points. First, this data includes both contributions and refunds (negative contribution amount):

In [ ]:
fec = fec[fec.contb_receipt_amt > 0]

Since Barack Obama and Mitt Romney are the main two candidates, we’ll also prepare a subset that just has contributions to their campaigns:

In [ ]:
fec_mrbo = fec[fec.cand_nm.isin(['Obama, Barack', 'Romney, Mitt'])]

*Example: 2012* >
### Donation Statistics by Occupation and Employer

Donations by occupation is another oft-studied statistic. First, the total number of donations by occupation is easy:

In [ ]:
fec.contbr_occupation.value_counts()[0:10]

We will notice by looking at the occupations that many refer to the same basic job type, or there are several variants of the same thing.

In [ ]:
occ_mapping = {'INFORMATION REQUESTED PER BEST EFFORTS' : 'NOT PROVIDED', 
               'INFORMATION REQUESTED' : 'NOT PROVIDED', 
               'INFORMATION REQUESTED (BEST EFFORTS)' : 'NOT PROVIDED', 
               'C.E.O.': 'CEO'}

In [ ]:
# If no mapping provided, return x
f = lambda x: occ_mapping.get(x, x) 
fec.contbr_occupation = fec.contbr_occupation.map(f)

We’ll also do the same thing for employers:

In [ ]:
emp_mapping = {'INFORMATION REQUESTED PER BEST EFFORTS' : 'NOT PROVIDED', 
               'INFORMATION REQUESTED' : 'NOT PROVIDED', 
               'SELF' : 'SELF-EMPLOYED', 
               'SELF EMPLOYED' : 'SELF-EMPLOYED'}

In [ ]:
# If no mapping provided, return x
f = lambda x: emp_mapping.get(x, x) 
fec.contbr_employer = fec.contbr_employer.map(f)

We can use **pivot_table** to aggregate the data by party and occupation, then filter down to the subset that donated at least $2 million overall:

In [ ]:
by_occupation = fec.pivot_table('contb_receipt_amt', 
                                index='contbr_occupation',
                                columns='party', aggfunc='sum')

In [ ]:
over_2mm = by_occupation[by_occupation.sum(1) > 2000000]

In [ ]:
over_2mm

It can be easier to look at this data graphically as a bar plot:

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
over_2mm.plot(kind = 'barh')

We might be interested in the top donor occupations or top companies donating to Obama and Romney.

In [ ]:
def get_top_amounts(group, key, n=5):
    totals = group.groupby(key)['contb_receipt_amt'].sum()
    
    # Order totals by key in descending order 
    return totals.sort_values(ascending=False)[:n]

In [ ]:
grouped = fec_mrbo.groupby('cand_nm')

In [ ]:
grouped.apply(get_top_amounts, 'contbr_employer', n=10)

*Example: 2012* >
### Bucketing Donation Amounts

A useful way to analyze this data is to use the cut function to discretize the contributor amounts into buckets by contribution size:

In [ ]:
bins = np.array([0, 1, 10, 100, 1000, 10000, 100000, 1000000, 10000000])

In [ ]:
labels = pd.cut(fec_mrbo.contb_receipt_amt, bins)

In [ ]:
grouped = fec_mrbo.groupby(['cand_nm', labels])

In [ ]:
grouped.size().unstack()

In [ ]:
bucket_sums = grouped.contb_receipt_amt.sum().unstack(0)

In [ ]:
bucket_sums

In [ ]:
normed_sums = bucket_sums.div(bucket_sums.sum(axis=1), axis=0)

In [ ]:
normed_sums

In [ ]:
normed_sums[:-2].plot(kind='barh', stacked=True)

#### Excercise

You could aggregate donations by donor name and zip code to adjust for donors who gave many small amounts versus one or more large donations.

*Example: 2012* >
### Donation Statistics by State

Aggregating the data by candidate and state is a routine affair:

In [ ]:
grouped = fec_mrbo.groupby(['cand_nm', 'contbr_st'])

In [ ]:
totals = grouped.contb_receipt_amt.sum().unstack(0).fillna(0)

In [ ]:
totals = totals[totals.sum(1) > 100000]

In [ ]:
totals[:10]

If we divide each row by the total contribution amount, we get the relative percentage of total donations by state for each candidate:

In [ ]:
percent = totals.div(totals.sum(1), axis=0)

In [ ]:
percent[:10]

#### Excercise: Plot this data on a map.

After locating a shape file for the state boundaries (http://nationalmap.gov/small_scale/atlasftp.html?openChapters=chpbound#chpbound) and learning a bit more about matplotlib and its basemap toolkit.

<img src='./Pics/picture_5_4.png'>
<img src='./Pics/picture_5_5.png'>

In [ ]:
!pip install --upgrade matplotlib

In [ ]:
!pip install pyshp

In [ ]:
!pip install GEOS

!conda install basemap  # in shell

In [ ]:
%matplotlib qt

In [ ]:
import mpl_toolkits

In [ ]:
from mpl_toolkits.basemap import Basemap

In [ ]:
from matplotlib import rcParams

In [ ]:
from matplotlib.collections import LineCollection

In [ ]:
import shapefile

In [ ]:
obama = percent['Obama, Barack']

In [ ]:
fig = plt.figure(figsize=(12, 12))

In [ ]:
ax = fig.add_axes([0.1,0.1,0.8,0.8])

In [ ]:
lllat = 21; urlat = 53; lllon = -118; urlon = -62

In [ ]:
m = Basemap(ax=ax, projection='stere',
            lon_0=(urlon + lllon) / 2, lat_0=(urlat + lllat) / 2, 
            llcrnrlat=lllat, urcrnrlat=urlat, llcrnrlon=lllon, urcrnrlon=urlon, resolution='l')
m.drawcoastlines() 
m.drawcountries()

In [ ]:
shp = shapefile.Reader('../Data/statesp020_nt00032/statesp020')

In [ ]:
shapes = shp.shapes()

In [ ]:
records = shp.records()

In [ ]:
state_to_code = {
        'Alaska': 'AK',
        'Alabama': 'AL',
        'Arkansas': 'AR',
        'American Samoa': 'AS',
        'Arizona': 'AZ',
        'California': 'CA',
        'Colorado': 'CO',
        'Connecticut': 'CT',
        'District of Columbia': 'DC',
        'Delaware': 'DE',
        'Florida': 'FL',
        'Georgia': 'GA',
        'Guam': 'GU',
        'Hawaii': 'HI',
        'Iowa': 'IA',
        'Idaho': 'ID',
        'Illinois': 'IL',
        'Indiana': 'IN',
        'Kansas': 'KS',
        'Kentucky': 'KY',
        'Louisiana': 'LA',
        'Massachusetts': 'MA',
        'Maryland': 'MD',
        'Maine': 'ME',
        'Michigan': 'MI',
        'Minnesota': 'MN',
        'Missouri': 'MO',
        'Northern Mariana Islands': 'MP',
        'Mississippi': 'MS',
        'Montana': 'MT',
        'National': 'NA',
        'North Carolina': 'NC',
        'North Dakota': 'ND',
        'Nebraska': 'NE',
        'New Hampshire': 'NH',
        'New Jersey': 'NJ',
        'New Mexico': 'NM',
        'Nevada': 'NV',
        'New York': 'NY',
        'Ohio': 'OH',
        'Oklahoma': 'OK',
        'Oregon': 'OR',
        'Pennsylvania': 'PA',
        'Puerto Rico': 'PR',
        'Rhode Island': 'RI',
        'South Carolina': 'SC',
        'South Dakota': 'SD',
        'Tennessee': 'TN',
        'Texas': 'TX',
        'Utah': 'UT',
        'Virginia': 'VA',
        'Virgin Islands': 'VI',
        'Vermont': 'VT',
        'Washington': 'WA',
        'Wisconsin': 'WI',
        'West Virginia': 'WV',
        'Wyoming': 'WY'
}

In [ ]:
obama.keys()

In [ ]:
for record, shape in zip(records, shapes):
    lons,lats = zip(*shape.points)
    data = np.array(m(lons, lats)).T
    
    try:
        per = obama[state_to_code[record[3]]]
    except KeyError:
        continue
    
    lines = LineCollection([data,], antialiaseds=(1,))
    lines.set_facecolors('k')
    lines.set_alpha(0.75 * per)
    lines.set_edgecolors('k')
    lines.set_linewidth(0.3)
    ax.add_collection(lines)